# Project Setup

In [1]:
import pandas as pd
import math
import numpy as np
from pathlib import Path

In [2]:
# path to the NSPL file
# download from https://geoportal.statistics.gov.uk/search?q=PRD_NSPL%20MAY_2024&sort=Date%20Created%7Ccreated%7Cdesc
nspl_path = 'data/NSPL21_MAY_2024_UK.csv'

# path to csv containing the postcodes to calucalte the distance between
data_to_process_path = 'data/postcodes_to_process.csv'

# path to store results
postcodes_with_distance_path = 'data/postcodes_with_distance.csv'

# path to store outliers
outliers_path = 'data/outliers.csv'

In [3]:


# Define the file path
file_path = Path(data_to_process_path)

# Check if the file exists
if file_path.exists():
    print(f"The file {file_path} exists.")
else:
    print(f"The file {file_path} does not exists.  Creating sample file")
    
    # create a file
    data = {
        'id': [1, 2, 3, 4, 5],
        'start_postcode': ['AB1 0AA', 'BD21 3RU', 'DD10 8DS', 'HP2 7QL', 'NE12 0XS'],
        'end_postcode': ['AB1 0AX', 'BD21 3SF', 'DD10 8DT', 'HP2 7QX', 'G84 9EP']
    }

    # Convert the dictionary into a pandas DataFrame
    df = pd.DataFrame(data)

    # write 
    df.to_csv(data_to_process_path, index=False)

The file data\postcodes_to_process.csv exists.


# Load Data

In [4]:
# only loading the postcode and northing and easting columns
columns_to_load = ['pcds', 'oseast1m', 'osnrth1m']
nspl_df = pd.read_csv(nspl_path, usecols=columns_to_load)

# Functions

In [5]:
def format_postcode(postcode):
    """
    Function to format a postcode.
    Remove any spaces and convert to uppercase.
    Insert a space the fourth character from the end.
    """
    cleaned_postcode = postcode.replace(" ", "").upper()
    formatted_postcode = cleaned_postcode[:-3] + ' ' + cleaned_postcode[-3:]
    return formatted_postcode

In [6]:
def get_postcode_coordinates(postcode, df):
    """
    Function to get northing and easting for a given postcode.
    Format postcode to uppercase with correct spacing.
    """
    formatted_postcode = format_postcode(postcode)
    result = df[df['pcds'] == formatted_postcode]

    if result.empty:
        return None, None

    northing = result['osnrth1m'].values[0]
    easting = result['oseast1m'].values[0]

    return northing, easting

In [7]:
def calculate_distance(start_pcode, end_pcode):
    """
    Function to calculate distance between two postcodes.
    """
    northing1, easting1 = get_postcode_coordinates(start_pcode, nspl_df)
    northing2, easting2 = get_postcode_coordinates(end_pcode, nspl_df)

    if northing1 is None or northing2 is None or easting1 is None or easting2 is None:
        return None  # Return None if any of the coordinates are not found

    # Calculate the deltas
    delta_easting = easting2 - easting1
    delta_northing = northing2 - northing1

    # Use Pythagorean theorem to calculate the distance
    distance = math.sqrt(delta_easting**2 + delta_northing**2)
    return distance

# Calculate the distance

In [8]:
# Load data to process
df = pd.read_csv(data_to_process_path)

# cacluate the distance
df['distance_meters'] = df.apply(lambda row: calculate_distance(row['start_postcode'], row['end_postcode']), axis=1)

# save the results to csv
df.to_csv(postcodes_with_distance_path, index=False)

# Display the DataFrame with the distance
print(df)

   id start_postcode end_postcode  distance_meters
0   1        AB1 0AA      AB1 0AX       713.431847
1   2       BD21 3RU     BD21 3SF       383.985677
2   3       DD10 8DS     DD10 8DT        71.867934
3   4        HP2 7QL      HP2 7QX       534.113284
4   5       NE12 0XS      G84 9EP    225088.078496


# Identify Outliers Using IQR Method

In [12]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['distance_meters'].quantile(0.25)
Q3 = df['distance_meters'].quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define the outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_df = df[(df['distance_meters'] < lower_bound) |
              (df['distance_meters'] > upper_bound)]

# store the results
outliers_df.to_csv(outliers_path, index=False)

# display the outliers
print('Outliers\n---------')
display(outliers_df)

Outliers
---------


,id,start_postcode,end_postcode,distance_meters
4,5,NE12 0XS,G84 9EP,225088.078496
